In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import talib as ta
import datetime
import vectorbt as vbt
import optuna
import hiplot as hip
import warnings

warnings.filterwarnings("ignore")

In [2]:
all_data = pd.read_csv('BTCUSDT_1h.csv')
all_data['open_time'] = pd.to_datetime(all_data['open_time'])

all_data = all_data.drop(['Unnamed: 0', 'close_time', 'ignore'], axis = 1)
all_data.columns = ['open_t', 'open', 'high', 'low', 'close', 'volume', 'USD_vol', 'trade_count', 'taker_buy_volume', 'taker_buy_USD_vol']

all_data.tail()

,open_t,open,high,low,close,volume,USD_vol,trade_count,taker_buy_volume,taker_buy_USD_vol
39331,2024-06-30 19:00:00,62057.4,62100.0,61887.0,62016.8,5533.263,3.430508e+08,69135,2587.240,1.604136e+08
39332,2024-06-30 20:00:00,62016.8,62399.0,61968.5,61981.6,10112.160,6.288445e+08,111277,5445.140,3.386765e+08
39333,2024-06-30 21:00:00,61981.7,62010.7,61825.4,61931.6,3431.207,2.124809e+08,51830,1778.343,1.101328e+08
39334,2024-06-30 22:00:00,61931.7,62955.3,61924.6,62894.9,19527.251,1.223194e+09,201464,10832.230,6.781865e+08
39335,2024-06-30 23:00:00,62894.9,63058.4,62753.9,62766.0,8430.946,5.304230e+08,106698,4112.704,2.588015e+08


In [3]:
# 取值到 2022 年 8 月 31 日 23:59:59 的数据作为 train_data
train_data = all_data[all_data['open_t'] <= '2022-08-31 23:59:59']

# 检查前几行数据
train_data.tail()


,open_t,open,high,low,close,volume,USD_vol,trade_count,taker_buy_volume,taker_buy_USD_vol
23275,2022-08-31 19:00:00,19993.9,20329.0,19981.0,20217.7,48753.548,9.839736e+08,275413,25624.333,5.171473e+08
23276,2022-08-31 20:00:00,20217.7,20232.7,20152.0,20186.6,13023.847,2.630068e+08,100092,6299.235,1.272185e+08
23277,2022-08-31 21:00:00,20186.5,20288.7,20150.1,20272.2,11100.402,2.243710e+08,90106,6127.146,1.238672e+08
23278,2022-08-31 22:00:00,20272.2,20285.6,20080.9,20148.2,23517.162,4.738536e+08,170843,11375.626,2.291987e+08
23279,2022-08-31 23:00:00,20148.3,20148.3,19959.8,20041.5,20369.711,4.079525e+08,153366,10421.438,2.086779e+08


In [4]:
test_data = all_data[all_data['open_t'] >= '2022-08-31 23:59:59']

test_data.tail()

,open_t,open,high,low,close,volume,USD_vol,trade_count,taker_buy_volume,taker_buy_USD_vol
39331,2024-06-30 19:00:00,62057.4,62100.0,61887.0,62016.8,5533.263,3.430508e+08,69135,2587.240,1.604136e+08
39332,2024-06-30 20:00:00,62016.8,62399.0,61968.5,61981.6,10112.160,6.288445e+08,111277,5445.140,3.386765e+08
39333,2024-06-30 21:00:00,61981.7,62010.7,61825.4,61931.6,3431.207,2.124809e+08,51830,1778.343,1.101328e+08
39334,2024-06-30 22:00:00,61931.7,62955.3,61924.6,62894.9,19527.251,1.223194e+09,201464,10832.230,6.781865e+08
39335,2024-06-30 23:00:00,62894.9,63058.4,62753.9,62766.0,8430.946,5.304230e+08,106698,4112.704,2.588015e+08


### Opt param

## bollin + ADX

In [ ]:
# import optuna
# import vectorbt as vbt
# import pandas as pd
# import talib as ta
# import numpy as np

# # Define the objective function for Optuna optimization
# def objective(trial):
#     # Suggest values for the parameters
#     bollinger_period = trial.suggest_int('bollinger_period', 12, 48, 4)
#     adx_period = trial.suggest_int('adx_period', 14, 40, 2)
#     adx_threshold = trial.suggest_int('adx_threshold', 20, 40, 2)
#     bollinger_std = trial.suggest_float('bollinger_std', 1.5, 3, step=0.2)
#     expansion_threshold = trial.suggest_float('expansion_threshold', 0.05, 0.2, step=0.01)  # New threshold parameter

#     # Calculate technical indicators
#     train_data['upper_band'], train_data['middle_band'], train_data['lower_band'] = ta.BBANDS(
#         train_data['close'], timeperiod=bollinger_period, nbdevup=bollinger_std, nbdevdn=bollinger_std)
#     train_data['ADX'] = ta.ADX(train_data['high'], train_data['low'], train_data['close'], timeperiod=adx_period)

#     # Calculate the expansion of the Bollinger Bands
#     train_data['band_expansion'] = (train_data['upper_band'] - train_data['lower_band']) / train_data['middle_band'].shift(1)

#     # Generate signals
#     sig = pd.DataFrame(index=train_data.index, columns=['long_entry', 'long_exit', 'short_entry', 'short_exit'])

#     # Long entry: Price crosses above the lower band
#     sig['long_entry'] = np.where(
#         (train_data['close'] > train_data['lower_band']) &
#         (train_data['close'].shift(1) <= train_data['lower_band']), 
#         0.5, 0
#     )

#     # Long add position: Price crosses above the middle band
#     sig['long_entry'] = np.where(
#         (train_data['close'] > train_data['middle_band']) &
#         (train_data['close'].shift(1) <= train_data['middle_band']),
#         1, sig['long_entry']
#     )

#     # Continue long: Price between the middle and upper band, with a bullish trend (ADX)
#     sig['long_entry'] = np.where(
#         (train_data['close'] > train_data['middle_band']) &
#         (train_data['close'] < train_data['upper_band']) &
#         (train_data['ADX'] > adx_threshold),
#         1, sig['long_entry']
#     )

#     # Long exit: Price crosses below the middle band
#     sig['long_exit'] = np.where(
#         (train_data['close'] < train_data['middle_band']) &
#         (train_data['close'].shift(1) >= train_data['middle_band']), 
#         1, 0
#     )

    
    
#     # Short entry: Price crosses below the upper band
#     sig['short_entry'] = np.where(
#         (train_data['close'] < train_data['upper_band']) &
#         (train_data['close'].shift(1) >= train_data['upper_band']), 
#         0.5, 0
#     )

#     # Short add position: Price crosses below the middle band
#     sig['short_entry'] = np.where(
#         (train_data['close'] < train_data['middle_band']) &
#         (train_data['close'].shift(1) >= train_data['middle_band']),
#         1, sig['short_entry']
#     )

#     # Continue short: Price between the middle and lower band, with a bearish trend (ADX)
#     sig['short_entry'] = np.where(
#         (train_data['close'] < train_data['middle_band']) &
#         (train_data['close'] > train_data['lower_band']) &
#         (train_data['ADX'] > adx_threshold),
#         1, sig['short_entry']
#     )

#     # Short exit: Price crosses above the middle band
#     sig['short_exit'] = np.where(
#         (train_data['close'] > train_data['middle_band']) &
#         (train_data['close'].shift(1) <= train_data['middle_band']), 
#         1, 0
#     )

#     # Build the portfolio and calculate statistics
#     pf1 = vbt.Portfolio.from_signals(
#         train_data['close'],
#         entries=sig['long_entry'],
#         exits=sig['long_exit'],
#         short_entries=sig['short_entry'],
#         short_exits=sig['short_exit'],
#         freq='h',
#         fees=0.0005,
#     )
    
#     # Calculate and return the total return
#     rtn = pf1.stats()
#     return rtn['Total Return [%]']

# # Run the Optuna optimization
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=600)

# # Retrieve and print the best parameters and value
# print(f"Best Parameters: {study.best_params}")
# print("Best value:", study.best_value)


In [ ]:
# # Retrieve best parameters from the Optuna study
# bollinger_period_p = study.best_params['bollinger_period']
# adx_period_p = study.best_params['adx_period']
# adx_threshold_p = study.best_params['adx_threshold']
# bollinger_std_p = study.best_params['bollinger_std']
# expansion_threshold_p = study.best_params['expansion_threshold']

# # Calculate technical indicators using the best parameters
# train_data['upper_band'], train_data['middle_band'], train_data['lower_band'] = ta.BBANDS(
#     train_data['close'], timeperiod=bollinger_period_p, nbdevup=bollinger_std_p, nbdevdn=bollinger_std_p)
# train_data['ADX'] = ta.ADX(train_data['high'], train_data['low'], train_data['close'], timeperiod=adx_period_p)

# # Calculate the expansion of the Bollinger Bands
# train_data['band_expansion'] = (train_data['upper_band'] - train_data['lower_band']) / train_data['middle_band'].shift(1)

# # Generate entry and exit signals with the band expansion threshold
# sig = pd.DataFrame(index=train_data.index)

# # Long entry (initial position): Price crosses above the lower band
# sig['long_entry'] = np.where(
#     (train_data['close'] > train_data['lower_band']) &
#     (train_data['close'].shift(1) <= train_data['lower_band']) &
#     (train_data['band_expansion'] < expansion_threshold_p),
#     0.5, 0
# )

# # Long add position: Price crosses above the middle band
# sig['long_entry'] = np.where(
#     (train_data['close'] > train_data['middle_band']) &
#     (train_data['close'].shift(1) <= train_data['middle_band']) |
#     (sig['long_entry'] == 1),
#     1, 0
# )

# # Continue long: Price between the middle and upper band, with a bullish trend (ADX)
# sig['long_entry'] = np.where(
#     (train_data['close'] > train_data['middle_band']) &
#     (train_data['close'] < train_data['upper_band']) &
#     (train_data['ADX'] > adx_threshold_p) |
#     (sig['long_entry'] == 1),
#     1, 0
# )

# # Long exit: Price crosses below the middle band
# sig['long_exit'] = np.where(
#     (train_data['close'] < train_data['middle_band']) &
#     (train_data['close'].shift(1) >= train_data['middle_band']),
#     1, 0
# )

# # Short entry: Price crosses below the middle band from above
# sig['short_entry'] = np.where(
#     (train_data['close'] < train_data['upper_band']) &
#     (train_data['close'].shift(1) >= train_data['upper_band']) &
#     (train_data['band_expansion'] < expansion_threshold_p),
#     0.5, 0
# )

# # Continue short: Price between the middle and lower band, with a bearish trend (ADX)
# sig['short_entry'] = np.where(
#     (train_data['close'] < train_data['middle_band']) &
#     (train_data['close'] > train_data['lower_band']) &
#     (train_data['ADX'] > adx_threshold_p) |
#     (sig['short_entry'] == 1),
#     1, 0
# )

# # Short exit: Price crosses above the middle band
# sig['short_exit'] = np.where(
#     (train_data['close'] > train_data['middle_band']) &
#     (train_data['close'].shift(1) <= train_data['middle_band']),
#     1, 0
# )

# # Create portfolio and calculate statistics
# pft = vbt.Portfolio.from_signals(
#     train_data['close'],
#     entries=sig['long_entry'],
#     exits=sig['long_exit'],
#     short_entries=sig['short_entry'],
#     short_exits=sig['short_exit'],
#     freq='h',
#     fees=0.0005
# )

# # Output portfolio statistics
# pft_stats = pft.stats()
# print(pft_stats)

# # Plotting the results
# fig = pft.plot(subplots=['cum_returns', 'orders', 'trade_pnl'], autosize=False, width=1400, height=600)
# fig.show()


In [ ]:
# # Retrieve best parameters from the Optuna study
# bollinger_period_p = study.best_params['bollinger_period']
# adx_period_p = study.best_params['adx_period']
# adx_threshold_p = study.best_params['adx_threshold']
# bollinger_std_p = study.best_params['bollinger_std']
# expansion_threshold_p = study.best_params['expansion_threshold']

# # Calculate technical indicators using the best parameters
# test_data['upper_band'], test_data['middle_band'], test_data['lower_band'] = ta.BBANDS(
#     test_data['close'], timeperiod=bollinger_period_p, nbdevup=bollinger_std_p, nbdevdn=bollinger_std_p)
# test_data['ADX'] = ta.ADX(test_data['high'], test_data['low'], test_data['close'], timeperiod=adx_period_p)

# # Calculate the expansion of the Bollinger Bands
# test_data['band_expansion'] = (test_data['upper_band'] - test_data['lower_band']) / test_data['middle_band'].shift(1)

# # Generate entry and exit signals with the band expansion threshold
# sig = pd.DataFrame(index=test_data.index)
# sig['long_entry'] = np.where(
#     (test_data['close'] < test_data['lower_band']) &
#     (test_data['ADX'] > adx_threshold_p) &
#     (test_data['band_expansion'] < expansion_threshold_p), 
#     1, 0)
# sig['long_exit'] = np.where(
#     test_data['close'] > test_data['upper_band'], 
#     1, 0)
# sig['short_entry'] = np.where(
#     (test_data['close'] > test_data['upper_band']) &
#     (test_data['ADX'] > adx_threshold_p) &
#     (test_data['band_expansion'] < expansion_threshold_p), 
#     1, 0)
# sig['short_exit'] = np.where(
#     test_data['close'] < test_data['lower_band'], 
#     1, 0)

# # Create portfolio and calculate statistics
# pft = vbt.Portfolio.from_signals(
#     test_data['close'],
#     entries=sig['long_entry'],
#     exits=sig['long_exit'],
#     short_entries=sig['short_entry'],
#     short_exits=sig['short_exit'],
#     freq='h',
#     fees=0.0005
# )

# # Output portfolio statistics
# pft_stats = pft.stats()
# print(pft_stats)

## MACD 

### 做多

In [5]:
import optuna
import vectorbt as vbt
import pandas as pd
import talib as ta
import numpy as np

# Define the objective function for Optuna optimization (Maximize Calmar Ratio)
def calmar_objective(trial):
    # Suggest values for the parameters
    macd_fast = trial.suggest_int('macd_fast', 12, 26)
    macd_slow = trial.suggest_int('macd_slow', 26, 50)
    macd_signal = trial.suggest_int('macd_signal', 9, 18)

    # Calculate MACD indicators
    train_data['MACD'], train_data['MACD_signal'], train_data['MACD_hist'] = ta.MACD(
        train_data['close'], fastperiod=macd_fast, slowperiod=macd_slow, signalperiod=macd_signal)

    # Generate signals for long positions
    sig = pd.DataFrame(index=train_data.index, columns=['long_entry', 'long_exit'])

    # Long entry: diff > 0
    sig['long_entry'] = np.where(
        (train_data['MACD'] - train_data['MACD_signal']) > 0, 
        1, 0
    )

    # Long exit: diff < 0
    sig['long_exit'] = np.where(
        (train_data['MACD'] - train_data['MACD_signal']) < 0, 
        1, 0
    )

    # Build the portfolio and calculate statistics
    pf1 = vbt.Portfolio.from_signals(
        train_data['close'],
        entries=sig['long_entry'],
        exits=sig['long_exit'],
        freq='h',
        fees=0.0004,
    )
    
    # Calculate necessary statistics manually
    total_return = pf1.total_return()
    max_drawdown = pf1.max_drawdown()

    # Ensure max_drawdown is not zero to avoid division by zero
    if max_drawdown == 0:
        calmar_ratio = 0
    else:
        # Calculate Calmar Ratio (Total Return / Max Drawdown)
        calmar_ratio = total_return / abs(max_drawdown)

    # Return the negative of Calmar Ratio to minimize the objective
    return -calmar_ratio

# Run the Optuna optimization to maximize Calmar Ratio
study_calmar = optuna.create_study(direction='minimize')
study_calmar.optimize(calmar_objective, n_trials=800)

# Retrieve and print the best parameters and value for maximizing Calmar Ratio
print(f"Best Parameters to Maximize Calmar Ratio: {study_calmar.best_params}")
print("Best value for Maximized Calmar Ratio:", -study_calmar.best_value)


[I 2024-08-21 12:25:07,086] A new study created in memory with name: no-name-3d6151bd-1c51-40f2-a5e5-3ac3d73679c6
[I 2024-08-21 12:25:10,549] Trial 0 finished with value: -4.309817472041517 and parameters: {'macd_fast': 19, 'macd_slow': 44, 'macd_signal': 14}. Best is trial 0 with value: -4.309817472041517.
[I 2024-08-21 12:25:10,563] Trial 1 finished with value: -4.5811069517785326 and parameters: {'macd_fast': 12, 'macd_slow': 29, 'macd_signal': 16}. Best is trial 1 with value: -4.5811069517785326.
[I 2024-08-21 12:25:10,578] Trial 2 finished with value: -4.175455643328723 and parameters: {'macd_fast': 22, 'macd_slow': 30, 'macd_signal': 18}. Best is trial 1 with value: -4.5811069517785326.
[I 2024-08-21 12:25:10,592] Trial 3 finished with value: -4.453610312634409 and parameters: {'macd_fast': 20, 'macd_slow': 29, 'macd_signal': 15}. Best is trial 1 with value: -4.5811069517785326.
[I 2024-08-21 12:25:10,609] Trial 4 finished with value: -2.9944749010950558 and parameters: {'macd_fa

Best Parameters to Maximize Calmar Ratio: {'macd_fast': 14, 'macd_slow': 49, 'macd_signal': 13}
Best value for Maximized Calmar Ratio: 6.823358494936641


### Train 

In [6]:
# Retrieve best parameters from the Optuna study for Long Strategy
macd_fast_p = study_calmar.best_params['macd_fast']  
macd_slow_p = study_calmar.best_params['macd_slow']
macd_signal_p = study_calmar.best_params['macd_signal']

# Calculate MACD indicators using the best parameters
train_data['MACD'], train_data['MACD_signal'], train_data['MACD_hist'] = ta.MACD(
    train_data['close'], fastperiod=macd_fast_p, slowperiod=macd_slow_p, signalperiod=macd_signal_p)

# Generate entry and exit signals for Long Strategy
sig = pd.DataFrame(index=train_data.index)
sig['long_entry'] = np.where(
    (train_data['MACD'] - train_data['MACD_signal']) > 0, 
    1, 0)
sig['long_exit'] = np.where(
    (train_data['MACD'] - train_data['MACD_signal']) < 0, 
    1, 0)

# Create portfolio and calculate statistics for Long Strategy
pft_long = vbt.Portfolio.from_signals(
    train_data['close'],
    entries=sig['long_entry'],
    exits=sig['long_exit'],
    freq='h',
    fees=0.0004
)

# Output portfolio statistics for Long Strategy
pft_long_stats = pft_long.stats()
print("Long Strategy Stats:")
print(pft_long_stats)


Long Strategy Stats:
Start                                                 0
End                                               23279
Period                                970 days 00:00:00
Start Value                                       100.0
End Value                                    395.302688
Total Return [%]                             295.302688
Benchmark Return [%]                         179.458416
Max Gross Exposure [%]                            100.0
Total Fees Paid                              185.224135
Max Drawdown [%]                              43.278202
Max Drawdown Duration                 532 days 03:00:00
Total Trades                                        597
Total Closed Trades                                 597
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  35.845896
Best Trade [%]                                20.572692
Worst Trade [%]            

### test 

In [7]:
# Retrieve best parameters from the Optuna study for Long Strategy
macd_fast_p = study_calmar.best_params['macd_fast']  
macd_slow_p = study_calmar.best_params['macd_slow']
macd_signal_p = study_calmar.best_params['macd_signal']

# Calculate MACD indicators using the best parameters
test_data['MACD'], test_data['MACD_signal'], test_data['MACD_hist'] = ta.MACD(
    test_data['close'], fastperiod=macd_fast_p, slowperiod=macd_slow_p, signalperiod=macd_signal_p)

# Generate entry and exit signals for Long Strategy
sig = pd.DataFrame(index=test_data.index)
sig['long_entry'] = np.where(
    (test_data['MACD'] - test_data['MACD_signal']) > 0, 
    1, 0)
sig['long_exit'] = np.where(
    (test_data['MACD'] - test_data['MACD_signal']) < 0, 
    1, 0)

# Create portfolio and calculate statistics for Long Strategy
pft_long = vbt.Portfolio.from_signals(
    test_data['close'],
    entries=sig['long_entry'],
    exits=sig['long_exit'],
    freq='h',
    fees=0.0004
)

# Output portfolio statistics for Long Strategy
pft_long_stats = pft_long.stats()
print("Long Strategy Stats:")
print(pft_long_stats)


Long Strategy Stats:
Start                                             23280
End                                               39335
Period                                669 days 00:00:00
Start Value                                       100.0
End Value                                    213.410214
Total Return [%]                             113.410214
Benchmark Return [%]                         212.935006
Max Gross Exposure [%]                            100.0
Total Fees Paid                               48.683142
Max Drawdown [%]                              20.788849
Max Drawdown Duration                 165 days 01:00:00
Total Trades                                        406
Total Closed Trades                                 405
Total Open Trades                                     1
Open Trade PnL                                 5.768536
Win Rate [%]                                  34.814815
Best Trade [%]                                21.201307
Worst Trade [%]            

### all data 

In [ ]:
# Retrieve best parameters from the Optuna study for Long Strategy
macd_fast_p = study_calmar.best_params['macd_fast']  
macd_slow_p = study_calmar.best_params['macd_slow']
macd_signal_p = study_calmar.best_params['macd_signal']

# Calculate MACD indicators using the best parameters
all_data['MACD'], all_data['MACD_signal'], all_data['MACD_hist'] = ta.MACD(
    all_data['close'], fastperiod=macd_fast_p, slowperiod=macd_slow_p, signalperiod=macd_signal_p)

# Generate entry and exit signals for Long Strategy
sig = pd.DataFrame(index=all_data.index)
sig['long_entry'] = np.where(
    (all_data['MACD'] - all_data['MACD_signal']) > 0, 
    1, 0)
sig['long_exit'] = np.where(
    (all_data['MACD'] - all_data['MACD_signal']) < 0, 
    1, 0)

# Create portfolio and calculate statistics for Long Strategy
pft_long = vbt.Portfolio.from_signals(
    all_data['close'],
    entries=sig['long_entry'],
    exits=sig['long_exit'],
    freq='h',
    fees=0.0004
)

# Output portfolio statistics for Long Strategy
pft_long_stats = pft_long.stats()
print("Long Strategy Stats:")
print(pft_long_stats)


### 加入volatility 濾網，並優化參數

In [ ]:
import optuna
import vectorbt as vbt
import pandas as pd
import talib as ta
import numpy as np

# Define the objective function for Optuna optimization (Maximize Calmar Ratio)
def calmar_objective(trial):
    # Suggest values for the parameters
    macd_fast = trial.suggest_int('macd_fast', 12, 26)
    macd_slow = trial.suggest_int('macd_slow', 26, 50)
    macd_signal = trial.suggest_int('macd_signal', 9, 18)
    atr_period = trial.suggest_int('atr_period', 14, 28)  # ATR period suggestion
    pctrank_window = trial.suggest_int('pctrank_window', 10, 50)  # PctRank window suggestion
    pctrank_threshold = trial.suggest_uniform('pctrank_threshold', 0.2, 0.8)  # PctRank threshold suggestion

    # Calculate MACD indicators
    train_data['MACD'], train_data['MACD_signal'], train_data['MACD_hist'] = ta.MACD(
        train_data['close'], fastperiod=macd_fast, slowperiod=macd_slow, signalperiod=macd_signal)

    # Calculate ATR (Average True Range)
    train_data['ATR'] = ta.ATR(train_data['high'], train_data['low'], train_data['close'], timeperiod=atr_period)

    # Calculate PctRank of ATR over a rolling window
    train_data['ATR_PctRank'] = train_data['ATR'].rolling(window=pctrank_window).apply(
        lambda x: pd.Series(x).rank(pct=True).iloc[-1])

    # Generate signals for long positions with volatility filter
    sig = pd.DataFrame(index=train_data.index, columns=['long_entry', 'long_exit'])

    # Long entry: diff > 0 and ATR_PctRank < pctrank_threshold
    sig['long_entry'] = np.where(
        ((train_data['MACD'] - train_data['MACD_signal']) > 0) & 
        (train_data['ATR_PctRank'] < pctrank_threshold), 
        1, 0
    )

    # Long exit: diff < 0
    sig['long_exit'] = np.where(
        (train_data['MACD'] - train_data['MACD_signal']) < 0, 
        1, 0
    )

    # Build the portfolio and calculate statistics
    pf1 = vbt.Portfolio.from_signals(
        train_data['close'],
        entries=sig['long_entry'],
        exits=sig['long_exit'],
        freq='h',
        fees=0.0004,
    )
    
    # Calculate necessary statistics manually
    total_return = pf1.total_return()
    max_drawdown = pf1.max_drawdown()

    # Ensure max_drawdown is not zero to avoid division by zero
    if max_drawdown == 0:
        calmar_ratio = 0
    else:
        # Calculate Calmar Ratio (Total Return / Max Drawdown)
        calmar_ratio = total_return / abs(max_drawdown)

    # Return the negative of Calmar Ratio to minimize the objective
    return -calmar_ratio

# Run the Optuna optimization to maximize Calmar Ratio
study_calmar = optuna.create_study(direction='minimize')
study_calmar.optimize(calmar_objective, n_trials=600)

# Retrieve and print the best parameters and value for maximizing Calmar Ratio
print(f"Best Parameters to Maximize Calmar Ratio: {study_calmar.best_params}")
print("Best value for Maximized Calmar Ratio:", -study_calmar.best_value)


### Train 

In [ ]:
# Retrieve best parameters from the Optuna study for Long Strategy
macd_fast_p = study_calmar.best_params['macd_fast']  
macd_slow_p = study_calmar.best_params['macd_slow']
macd_signal_p = study_calmar.best_params['macd_signal']
atr_period_p = study_calmar.best_params['atr_period']
pctrank_window_p = study_calmar.best_params['pctrank_window']
pctrank_threshold_p = study_calmar.best_params['pctrank_threshold']

# Calculate MACD indicators using the best parameters
train_data['MACD'], train_data['MACD_signal'], train_data['MACD_hist'] = ta.MACD(
    train_data['close'], fastperiod=macd_fast_p, slowperiod=macd_slow_p, signalperiod=macd_signal_p)

# Calculate ATR (Average True Range) using the best parameters
train_data['ATR'] = ta.ATR(train_data['high'], train_data['low'], train_data['close'], timeperiod=atr_period_p)

# Calculate PctRank of ATR over the optimized rolling window
train_data['ATR_PctRank'] = train_data['ATR'].rolling(window=pctrank_window_p).apply(
    lambda x: pd.Series(x).rank(pct=True).iloc[-1])

# Generate entry and exit signals for Long Strategy with the PctRank filter
sig = pd.DataFrame(index=train_data.index)
sig['long_entry'] = np.where(
    ((train_data['MACD'] - train_data['MACD_signal']) > 0) & 
    (train_data['ATR_PctRank'] < pctrank_threshold_p), 
    1, 0
)
sig['long_exit'] = np.where(
    (train_data['MACD'] - train_data['MACD_signal']) < 0, 
    1, 0
)

# Create portfolio and calculate statistics for Long Strategy
pft_long = vbt.Portfolio.from_signals(
    train_data['close'],
    entries=sig['long_entry'],
    exits=sig['long_exit'],
    freq='h',
    fees=0.0004
)

# Output portfolio statistics for Long Strategy
pft_long_stats = pft_long.stats()
print("Long Strategy Stats:")
print(pft_long_stats)


### Test

In [ ]:
# Retrieve best parameters from the Optuna study for Long Strategy
macd_fast_p = study_calmar.best_params['macd_fast']  
macd_slow_p = study_calmar.best_params['macd_slow']
macd_signal_p = study_calmar.best_params['macd_signal']
atr_period_p = study_calmar.best_params['atr_period']
pctrank_window_p = study_calmar.best_params['pctrank_window']
pctrank_threshold_p = study_calmar.best_params['pctrank_threshold']

# Calculate MACD indicators using the best parameters
test_data['MACD'], test_data['MACD_signal'], test_data['MACD_hist'] = ta.MACD(
    test_data['close'], fastperiod=macd_fast_p, slowperiod=macd_slow_p, signalperiod=macd_signal_p)

# Calculate ATR (Average True Range) using the best parameters
test_data['ATR'] = ta.ATR(test_data['high'], test_data['low'], test_data['close'], timeperiod=atr_period_p)

# Calculate PctRank of ATR over the optimized rolling window
test_data['ATR_PctRank'] = test_data['ATR'].rolling(window=pctrank_window_p).apply(
    lambda x: pd.Series(x).rank(pct=True).iloc[-1])

# Generate entry and exit signals for Long Strategy with the PctRank filter
sig = pd.DataFrame(index=test_data.index)
sig['long_entry'] = np.where(
    ((test_data['MACD'] - test_data['MACD_signal']) > 0) & 
    (test_data['ATR_PctRank'] < pctrank_threshold_p), 
    1, 0
)
sig['long_exit'] = np.where(
    (test_data['MACD'] - test_data['MACD_signal']) < 0, 
    1, 0
)

# Create portfolio and calculate statistics for Long Strategy
pft_long = vbt.Portfolio.from_signals(
    test_data['close'],
    entries=sig['long_entry'],
    exits=sig['long_exit'],
    freq='h',
    fees=0.0004
)

# Output portfolio statistics for Long Strategy
pft_long_stats = pft_long.stats()
print("Long Strategy Stats:")
print(pft_long_stats)


### ALL 

In [ ]:
# Retrieve best parameters from the Optuna study for Long Strategy
macd_fast_p = study_calmar.best_params['macd_fast']  
macd_slow_p = study_calmar.best_params['macd_slow']
macd_signal_p = study_calmar.best_params['macd_signal']
atr_period_p = study_calmar.best_params['atr_period']
pctrank_window_p = study_calmar.best_params['pctrank_window']
pctrank_threshold_p = study_calmar.best_params['pctrank_threshold']

# Calculate MACD indicators using the best parameters
all_data['MACD'], all_data['MACD_signal'], all_data['MACD_hist'] = ta.MACD(
    all_data['close'], fastperiod=macd_fast_p, slowperiod=macd_slow_p, signalperiod=macd_signal_p)

# Calculate ATR (Average True Range) using the best parameters
all_data['ATR'] = ta.ATR(all_data['high'], all_data['low'], all_data['close'], timeperiod=atr_period_p)

# Calculate PctRank of ATR over the optimized rolling window
all_data['ATR_PctRank'] = all_data['ATR'].rolling(window=pctrank_window_p).apply(
    lambda x: pd.Series(x).rank(pct=True).iloc[-1])

# Generate entry and exit signals for Long Strategy with the PctRank filter
sig = pd.DataFrame(index=all_data.index)
sig['long_entry'] = np.where(
    ((all_data['MACD'] - all_data['MACD_signal']) > 0) & 
    (all_data['ATR_PctRank'] < pctrank_threshold_p), 
    1, 0
)
sig['long_exit'] = np.where(
    (all_data['MACD'] - all_data['MACD_signal']) < 0, 
    1, 0
)

# Create portfolio and calculate statistics for Long Strategy
pft_long = vbt.Portfolio.from_signals(
    all_data['close'],
    entries=sig['long_entry'],
    exits=sig['long_exit'],
    freq='h',
    fees=0.0004
)

# Output portfolio statistics for Long Strategy
pft_long_stats = pft_long.stats()
print("Long Strategy Stats:")
print(pft_long_stats)


### Optimize : Min - Sharpe 

In [ ]:
import optuna
import vectorbt as vbt
import pandas as pd
import talib as ta
import numpy as np

# Define the objective function for Optuna optimization (Maximize Sharpe Ratio)
def sharpe_objective(trial):
    # Suggest values for the parameters
    macd_fast = trial.suggest_int('macd_fast', 12, 26)
    macd_slow = trial.suggest_int('macd_slow', 26, 50)
    macd_signal = trial.suggest_int('macd_signal', 9, 18)
    atr_period = trial.suggest_int('atr_period', 14, 28)  # ATR period suggestion
    pctrank_window = trial.suggest_int('pctrank_window', 10, 50)  # PctRank window suggestion
    pctrank_threshold = trial.suggest_uniform('pctrank_threshold', 0.2, 0.8)  # PctRank threshold suggestion

    # Calculate MACD indicators
    train_data['MACD'], train_data['MACD_signal'], train_data['MACD_hist'] = ta.MACD(
        train_data['close'], fastperiod=macd_fast, slowperiod=macd_slow, signalperiod=macd_signal)

    # Calculate ATR (Average True Range)
    train_data['ATR'] = ta.ATR(train_data['high'], train_data['low'], train_data['close'], timeperiod=atr_period)

    # Calculate PctRank of ATR over a rolling window
    train_data['ATR_PctRank'] = train_data['ATR'].rolling(window=pctrank_window).apply(
        lambda x: pd.Series(x).rank(pct=True).iloc[-1])

    # Generate signals for long positions with volatility filter
    sig = pd.DataFrame(index=train_data.index, columns=['long_entry', 'long_exit'])

    # Long entry: diff > 0 and ATR_PctRank < pctrank_threshold
    sig['long_entry'] = np.where(
        ((train_data['MACD'] - train_data['MACD_signal']) > 0) & 
        (train_data['ATR_PctRank'] < pctrank_threshold), 
        1, 0
    )

    # Long exit: diff < 0
    sig['long_exit'] = np.where(
        (train_data['MACD'] - train_data['MACD_signal']) < 0, 
        1, 0
    )

    # Build the portfolio and calculate statistics
    pf1 = vbt.Portfolio.from_signals(
        train_data['close'],
        entries=sig['long_entry'],
        exits=sig['long_exit'],
        freq='h',
        fees=0.0004,
    )
    
    # Calculate necessary statistics manually
    total_return = pf1.total_return()
    
    # Calculate daily returns and annualized volatility
    daily_returns = pf1.returns()
    annual_volatility = daily_returns.std() * np.sqrt(365)  

    # Ensure annual_volatility is not zero to avoid division by zero
    if annual_volatility == 0:
        sharpe_ratio = 0
    else:
        # Calculate Sharpe Ratio (Total Return / Annual Volatility)
        sharpe_ratio = total_return / annual_volatility

    # Return the negative of Sharpe Ratio to minimize the objective
    return -sharpe_ratio

# Run the Optuna optimization to maximize Sharpe Ratio
study_sharpe = optuna.create_study(direction='minimize')
study_sharpe.optimize(sharpe_objective, n_trials=700)

# Retrieve and print the best parameters and value for maximizing Sharpe Ratio
print(f"Best Parameters to Maximize Sharpe Ratio: {study_sharpe.best_params}")
print("Best value for Maximized Sharpe Ratio:", -study_sharpe.best_value)


### train 

In [ ]:
# Retrieve best parameters from the Optuna study for Long Strategy
macd_fast_p = study_sharpe.best_params['macd_fast']  
macd_slow_p = study_sharpe.best_params['macd_slow']
macd_signal_p = study_sharpe.best_params['macd_signal']
atr_period_p = study_sharpe.best_params['atr_period']
pctrank_window_p = study_sharpe.best_params['pctrank_window']
pctrank_threshold_p = study_sharpe.best_params['pctrank_threshold']

# Calculate MACD indicators using the best parameters
train_data['MACD'], train_data['MACD_signal'], train_data['MACD_hist'] = ta.MACD(
    train_data['close'], fastperiod=macd_fast_p, slowperiod=macd_slow_p, signalperiod=macd_signal_p)

# Calculate ATR (Average True Range) using the best parameters
train_data['ATR'] = ta.ATR(train_data['high'], train_data['low'], train_data['close'], timeperiod=atr_period_p)

# Calculate PctRank of ATR over the optimized rolling window
train_data['ATR_PctRank'] = train_data['ATR'].rolling(window=pctrank_window_p).apply(
    lambda x: pd.Series(x).rank(pct=True).iloc[-1])

# Generate entry and exit signals for Long Strategy with the PctRank filter
sig = pd.DataFrame(index=train_data.index)
sig['long_entry'] = np.where(
    ((train_data['MACD'] - train_data['MACD_signal']) > 0) & 
    (train_data['ATR_PctRank'] < pctrank_threshold_p), 
    1, 0
)
sig['long_exit'] = np.where(
    (train_data['MACD'] - train_data['MACD_signal']) < 0, 
    1, 0
)

# Create portfolio and calculate statistics for Long Strategy
pft_long = vbt.Portfolio.from_signals(
    train_data['close'],
    entries=sig['long_entry'],
    exits=sig['long_exit'],
    freq='h',
    fees=0.0004
)

# Output portfolio statistics for Long Strategy
pft_long_stats = pft_long.stats()
print("Long Strategy Stats:")
print(pft_long_stats)


### Test 

In [ ]:
# Retrieve best parameters from the Optuna study for Long Strategy
macd_fast_p = study_sharpe.best_params['macd_fast']  
macd_slow_p = study_sharpe.best_params['macd_slow']
macd_signal_p = study_sharpe.best_params['macd_signal']
atr_period_p = study_sharpe.best_params['atr_period']
pctrank_window_p = study_sharpe.best_params['pctrank_window']
pctrank_threshold_p = study_sharpe.best_params['pctrank_threshold']

# Calculate MACD indicators using the best parameters
test_data['MACD'], test_data['MACD_signal'], test_data['MACD_hist'] = ta.MACD(
    test_data['close'], fastperiod=macd_fast_p, slowperiod=macd_slow_p, signalperiod=macd_signal_p)

# Calculate ATR (Average True Range) using the best parameters
test_data['ATR'] = ta.ATR(test_data['high'], test_data['low'], test_data['close'], timeperiod=atr_period_p)

# Calculate PctRank of ATR over the optimized rolling window
test_data['ATR_PctRank'] = test_data['ATR'].rolling(window=pctrank_window_p).apply(
    lambda x: pd.Series(x).rank(pct=True).iloc[-1])

# Generate entry and exit signals for Long Strategy with the PctRank filter
sig = pd.DataFrame(index=test_data.index)
sig['long_entry'] = np.where(
    ((test_data['MACD'] - test_data['MACD_signal']) > 0) & 
    (test_data['ATR_PctRank'] < pctrank_threshold_p), 
    1, 0
)
sig['long_exit'] = np.where(
    (test_data['MACD'] - test_data['MACD_signal']) < 0, 
    1, 0
)

# Create portfolio and calculate statistics for Long Strategy
pft_long = vbt.Portfolio.from_signals(
    test_data['close'],
    entries=sig['long_entry'],
    exits=sig['long_exit'],
    freq='h',
    fees=0.0004
)

# Output portfolio statistics for Long Strategy
pft_long_stats = pft_long.stats()
print("Long Strategy Stats:")
print(pft_long_stats)


### All data 

In [ ]:
# Retrieve best parameters from the Optuna study for Long Strategy
macd_fast_p = study_sharpe.best_params['macd_fast']  
macd_slow_p = study_sharpe.best_params['macd_slow']
macd_signal_p = study_sharpe.best_params['macd_signal']
atr_period_p = study_sharpe.best_params['atr_period']
pctrank_window_p = study_sharpe.best_params['pctrank_window']
pctrank_threshold_p = study_sharpe.best_params['pctrank_threshold']

# Calculate MACD indicators using the best parameters
all_data['MACD'], all_data['MACD_signal'], all_data['MACD_hist'] = ta.MACD(
    all_data['close'], fastperiod=macd_fast_p, slowperiod=macd_slow_p, signalperiod=macd_signal_p)

# Calculate ATR (Average True Range) using the best parameters
all_data['ATR'] = ta.ATR(all_data['high'], all_data['low'], all_data['close'], timeperiod=atr_period_p)

# Calculate PctRank of ATR over the optimized rolling window
all_data['ATR_PctRank'] = all_data['ATR'].rolling(window=pctrank_window_p).apply(
    lambda x: pd.Series(x).rank(pct=True).iloc[-1])

# Generate entry and exit signals for Long Strategy with the PctRank filter
sig = pd.DataFrame(index=all_data.index)
sig['long_entry'] = np.where(
    ((all_data['MACD'] - all_data['MACD_signal']) > 0) & 
    (all_data['ATR_PctRank'] < pctrank_threshold_p), 
    1, 0
)
sig['long_exit'] = np.where(
    (all_data['MACD'] - all_data['MACD_signal']) < 0, 
    1, 0
)

# Create portfolio and calculate statistics for Long Strategy
pft_long = vbt.Portfolio.from_signals(
    all_data['close'],
    entries=sig['long_entry'],
    exits=sig['long_exit'],
    freq='h',
    fees=0.0004
)

# Output portfolio statistics for Long Strategy
pft_long_stats = pft_long.stats()
print("Long Strategy Stats:")
print(pft_long_stats)


### Plot 

In [ ]:
# Plot the strategy using vectorbt's built-in plotting capabilities
fig = pft_long.plot(
    subplots=['cum_returns', 'orders', 'trade_pnl'],  # Specify the subplots to include
    autosize=False,  # Disable autosizing if you want to specify custom size
    width=2000,  # Width of the plot
    height=1200  # Height of the plot
)

# Show the plot
fig.show()

### Max Sharpe 

In [ ]:
import optuna
import vectorbt as vbt
import pandas as pd
import talib as ta
import numpy as np

# Define the objective function for Optuna optimization (Maximize Sharpe Ratio)
def sharpe_objective(trial):
    # Suggest values for the parameters
    macd_fast = trial.suggest_int('macd_fast', 12, 26)
    macd_slow = trial.suggest_int('macd_slow', 26, 50)
    macd_signal = trial.suggest_int('macd_signal', 9, 18)
    atr_period = trial.suggest_int('atr_period', 14, 28)  # ATR period suggestion
    pctrank_window = trial.suggest_int('pctrank_window', 10, 50)  # PctRank window suggestion
    pctrank_threshold = trial.suggest_uniform('pctrank_threshold', 0.2, 0.8)  # PctRank threshold suggestion
    # Calculate MACD indicators
    train_data['MACD'], train_data['MACD_signal'], train_data['MACD_hist'] = ta.MACD(
        train_data['close'], fastperiod=macd_fast, slowperiod=macd_slow, signalperiod=macd_signal)

    # Calculate ATR (Average True Range)
    train_data['ATR'] = ta.ATR(train_data['high'], train_data['low'], train_data['close'], timeperiod=atr_period)

    # Calculate PctRank of ATR over a rolling window
    train_data['ATR_PctRank'] = train_data['ATR'].rolling(window=pctrank_window).apply(
        lambda x: pd.Series(x).rank(pct=True).iloc[-1])

    # Generate signals for long positions with volatility filter
    sig = pd.DataFrame(index=train_data.index, columns=['long_entry', 'long_exit'])

    # Long entry: diff > 0 and ATR_PctRank < pctrank_threshold
    sig['long_entry'] = np.where(
        ((train_data['MACD'] - train_data['MACD_signal']) > 0) & 
        (train_data['ATR_PctRank'] < pctrank_threshold), 
        1, 0
    )

    # Long exit: diff < 0
    sig['long_exit'] = np.where(
        (train_data['MACD'] - train_data['MACD_signal']) < 0, 
        1, 0
    )

    # Build the portfolio and calculate statistics
    pf1 = vbt.Portfolio.from_signals(
        train_data['close'],
        entries=sig['long_entry'],
        exits=sig['long_exit'],
        freq='h',
        fees=0.0004,
    )
    
    # Calculate necessary statistics manually
    total_return = pf1.total_return()
    
    # Calculate daily returns and annualized volatility
    daily_returns = pf1.returns()
    annual_volatility = daily_returns.std() * np.sqrt(365)  # Assuming 252 trading days in a year

    # Ensure annual_volatility is not zero to avoid division by zero
    if annual_volatility == 0:
        sharpe_ratio = 0
    else:
        # Calculate Sharpe Ratio (Total Return / Annual Volatility)
        sharpe_ratio = total_return / annual_volatility

    # Return the Sharpe Ratio to maximize the objective
    return sharpe_ratio

# Run the Optuna optimization to maximize Sharpe Ratio
study_sharpe = optuna.create_study(direction='maximize')
study_sharpe.optimize(sharpe_objective, n_trials=500)

# Retrieve and print the best parameters and value for maximizing Sharpe Ratio
print(f"Best Parameters to Maximize Sharpe Ratio: {study_sharpe.best_params}")
print("Best value for Maximized Sharpe Ratio:", study_sharpe.best_value)


In [ ]:
# Retrieve best parameters from the Optuna study for Long Strategy
macd_fast_p = study_sharpe.best_params['macd_fast']  
macd_slow_p = study_sharpe.best_params['macd_slow']
macd_signal_p = study_sharpe.best_params['macd_signal']
atr_period_p = study_sharpe.best_params['atr_period']
pctrank_window_p = study_sharpe.best_params['pctrank_window']
pctrank_threshold_p = study_sharpe.best_params['pctrank_threshold']

# Calculate MACD indicators using the best parameters
train_data['MACD'], train_data['MACD_signal'], train_data['MACD_hist'] = ta.MACD(
    train_data['close'], fastperiod=macd_fast_p, slowperiod=macd_slow_p, signalperiod=macd_signal_p)

# Calculate ATR (Average True Range) using the best parameters
train_data['ATR'] = ta.ATR(train_data['high'], train_data['low'], train_data['close'], timeperiod=atr_period_p)

# Calculate PctRank of ATR over the optimized rolling window
train_data['ATR_PctRank'] = train_data['ATR'].rolling(window=pctrank_window_p).apply(
    lambda x: pd.Series(x).rank(pct=True).iloc[-1])

# Generate entry and exit signals for Long Strategy with the PctRank filter
sig = pd.DataFrame(index=train_data.index)
sig['long_entry'] = np.where(
    ((train_data['MACD'] - train_data['MACD_signal']) > 0) & 
    (train_data['ATR_PctRank'] < pctrank_threshold_p), 
    1, 0
)
sig['long_exit'] = np.where(
    (train_data['MACD'] - train_data['MACD_signal']) < 0, 
    1, 0
)

# Create portfolio and calculate statistics for Long Strategy
pft_long = vbt.Portfolio.from_signals(
    train_data['close'],
    entries=sig['long_entry'],
    exits=sig['long_exit'],
    freq='h',
    fees=0.0004
)

# Output portfolio statistics for Long Strategy
pft_long_stats = pft_long.stats()
print("Long Strategy Stats:")
print(pft_long_stats)


In [ ]:
# Retrieve best parameters from the Optuna study for Long Strategy
macd_fast_p = study_sharpe.best_params['macd_fast']  
macd_slow_p = study_sharpe.best_params['macd_slow']
macd_signal_p = study_sharpe.best_params['macd_signal']
atr_period_p = study_sharpe.best_params['atr_period']
pctrank_window_p = study_sharpe.best_params['pctrank_window']
pctrank_threshold_p = study_sharpe.best_params['pctrank_threshold']

# Calculate MACD indicators using the best parameters
test_data['MACD'], test_data['MACD_signal'], test_data['MACD_hist'] = ta.MACD(
    test_data['close'], fastperiod=macd_fast_p, slowperiod=macd_slow_p, signalperiod=macd_signal_p)

# Calculate ATR (Average True Range) using the best parameters
test_data['ATR'] = ta.ATR(test_data['high'], test_data['low'], test_data['close'], timeperiod=atr_period_p)

# Calculate PctRank of ATR over the optimized rolling window
test_data['ATR_PctRank'] = test_data['ATR'].rolling(window=pctrank_window_p).apply(
    lambda x: pd.Series(x).rank(pct=True).iloc[-1])

# Generate entry and exit signals for Long Strategy with the PctRank filter
sig = pd.DataFrame(index=test_data.index)
sig['long_entry'] = np.where(
    ((test_data['MACD'] - test_data['MACD_signal']) > 0) & 
    (test_data['ATR_PctRank'] < pctrank_threshold_p), 
    1, 0
)
sig['long_exit'] = np.where(
    (test_data['MACD'] - test_data['MACD_signal']) < 0, 
    1, 0
)

# Create portfolio and calculate statistics for Long Strategy
pft_long = vbt.Portfolio.from_signals(
    test_data['close'],
    entries=sig['long_entry'],
    exits=sig['long_exit'],
    freq='h',
    fees=0.0004
)

# Output portfolio statistics for Long Strategy
pft_long_stats = pft_long.stats()
print("Long Strategy Stats:")
print(pft_long_stats)


In [ ]:
# Retrieve best parameters from the Optuna study for Long Strategy
macd_fast_p = study_sharpe.best_params['macd_fast']  
macd_slow_p = study_sharpe.best_params['macd_slow']
macd_signal_p = study_sharpe.best_params['macd_signal']
atr_period_p = study_sharpe.best_params['atr_period']
pctrank_window_p = study_sharpe.best_params['pctrank_window']
pctrank_threshold_p = study_sharpe.best_params['pctrank_threshold']

# Calculate MACD indicators using the best parameters
all_data['MACD'], all_data['MACD_signal'], all_data['MACD_hist'] = ta.MACD(
    all_data['close'], fastperiod=macd_fast_p, slowperiod=macd_slow_p, signalperiod=macd_signal_p)

# Calculate ATR (Average True Range) using the best parameters
all_data['ATR'] = ta.ATR(all_data['high'], all_data['low'], all_data['close'], timeperiod=atr_period_p)

# Calculate PctRank of ATR over the optimized rolling window
all_data['ATR_PctRank'] = all_data['ATR'].rolling(window=pctrank_window_p).apply(
    lambda x: pd.Series(x).rank(pct=True).iloc[-1])

# Generate entry and exit signals for Long Strategy with the PctRank filter
sig = pd.DataFrame(index=all_data.index)
sig['long_entry'] = np.where(
    ((all_data['MACD'] - all_data['MACD_signal']) > 0) & 
    (all_data['ATR_PctRank'] < pctrank_threshold_p), 
    1, 0
)
sig['long_exit'] = np.where(
    (all_data['MACD'] - all_data['MACD_signal']) < 0, 
    1, 0
)

# Create portfolio and calculate statistics for Long Strategy
pft_long = vbt.Portfolio.from_signals(
    all_data['close'],
    entries=sig['long_entry'],
    exits=sig['long_exit'],
    freq='h',
    fees=0.0004
)

# Output portfolio statistics for Long Strategy
pft_long_stats = pft_long.stats()
print("Long Strategy Stats:")
print(pft_long_stats)


In [ ]:
# Plot the strategy using vectorbt's built-in plotting capabilities
fig = pft_long.plot(
    subplots=['cum_returns', 'orders', 'trade_pnl'],  # Specify the subplots to include
    autosize=False,  # Disable autosizing if you want to specify custom size
    width=1600,  # Width of the plot
    height=1200  # Height of the plot
)

# Show the plot
fig.show()

### MACD + Volatility 濾網 + ADX ( Max Sharpe )

In [ ]:
import optuna
import vectorbt as vbt
import pandas as pd
import talib as ta
import numpy as np

# Define the objective function for Optuna optimization (Maximize Sharpe Ratio)
def sharpe_objective(trial):
    # Suggest values for the parameters
    macd_fast = trial.suggest_int('macd_fast', 12, 26)
    macd_slow = trial.suggest_int('macd_slow', 26, 50)
    macd_signal = trial.suggest_int('macd_signal', 9, 18)
    atr_period = trial.suggest_int('atr_period', 14, 28)  # ATR period suggestion
    pctrank_window = trial.suggest_int('pctrank_window', 10, 50)  # PctRank window suggestion
    pctrank_threshold = trial.suggest_uniform('pctrank_threshold', 0.2, 0.8)  # PctRank threshold suggestion
    adx_period = trial.suggest_int('adx_period', 10, 30)  # ADX period suggestion
    adx_threshold = trial.suggest_uniform('adx_threshold', 20, 40)  # ADX threshold suggestion
    
    # Calculate MACD indicators
    train_data['MACD'], train_data['MACD_signal'], train_data['MACD_hist'] = ta.MACD(
        train_data['close'], fastperiod=macd_fast, slowperiod=macd_slow, signalperiod=macd_signal)

    # Calculate ATR (Average True Range)
    train_data['ATR'] = ta.ATR(train_data['high'], train_data['low'], train_data['close'], timeperiod=atr_period)

    # Calculate ADX (Average Directional Index)
    train_data['ADX'] = ta.ADX(train_data['high'], train_data['low'], train_data['close'], timeperiod=adx_period)

    # Calculate PctRank of ATR over a rolling window
    train_data['ATR_PctRank'] = train_data['ATR'].rolling(window=pctrank_window).apply(
        lambda x: pd.Series(x).rank(pct=True).iloc[-1])

    # Generate signals for long positions with volatility and ADX filter
    sig = pd.DataFrame(index=train_data.index, columns=['long_entry', 'long_exit'])

    # Long entry: MACD conditions, ATR_PctRank < threshold, and ADX > threshold
    sig['long_entry'] = np.where(
        ((train_data['MACD'] - train_data['MACD_signal']) > 0) & 
        (train_data['ATR_PctRank'] < pctrank_threshold) & 
        (train_data['ADX'] > adx_threshold), 
        1, 0
    )

    # Long exit: MACD signal crosses down
    sig['long_exit'] = np.where(
        (train_data['MACD'] - train_data['MACD_signal']) < 0, 
        1, 0
    )

    # Build the portfolio and calculate statistics
    pf1 = vbt.Portfolio.from_signals(
        train_data['close'],
        entries=sig['long_entry'],
        exits=sig['long_exit'],
        freq='h',
        fees=0.0004,
    )
    
    # Calculate necessary statistics manually
    total_return = pf1.total_return()
    
    # Calculate daily returns and annualized volatility
    daily_returns = pf1.returns()
    annual_volatility = daily_returns.std() * np.sqrt(365)  # Adjust based on actual trading days

    # Ensure annual_volatility is not zero to avoid division by zero
    if annual_volatility == 0:
        sharpe_ratio = 0
    else:
        # Calculate Sharpe Ratio (Total Return / Annual Volatility)
        sharpe_ratio = total_return / annual_volatility

    # Return the Sharpe Ratio to maximize the objective
    return sharpe_ratio

# Run the Optuna optimization to maximize Sharpe Ratio
study_sharpe = optuna.create_study(direction='maximize')
study_sharpe.optimize(sharpe_objective, n_trials=400)

# Retrieve and print the best parameters and value for maximizing Sharpe Ratio
print(f"Best Parameters to Maximize Sharpe Ratio: {study_sharpe.best_params}")
print("Best value for Maximized Sharpe Ratio:", study_sharpe.best_value)


In [ ]:
# Retrieve best parameters from the Optuna study for Long Strategy
macd_fast_p = study_calmar.best_params['macd_fast']  
macd_slow_p = study_calmar.best_params['macd_slow']
macd_signal_p = study_calmar.best_params['macd_signal']
atr_period_p = study_calmar.best_params['atr_period']
pctrank_window_p = study_calmar.best_params['pctrank_window']
pctrank_threshold_p = study_calmar.best_params['pctrank_threshold']
adx_period_p = study_calmar.best_params['adx_period']
adx_threshold_p = study_calmar.best_params['adx_threshold']

# Calculate MACD indicators using the best parameters
train_data['MACD'], train_data['MACD_signal'], train_data['MACD_hist'] = ta.MACD(
    train_data['close'], fastperiod=macd_fast_p, slowperiod=macd_slow_p, signalperiod=macd_signal_p)

# Calculate ATR (Average True Range) using the best parameters
train_data['ATR'] = ta.ATR(train_data['high'], train_data['low'], train_data['close'], timeperiod=atr_period_p)

# Calculate ADX (Average Directional Index) using the best parameters
train_data['ADX'] = ta.ADX(train_data['high'], train_data['low'], train_data['close'], timeperiod=adx_period_p)

# Calculate PctRank of ATR over the optimized rolling window
train_data['ATR_PctRank'] = train_data['ATR'].rolling(window=pctrank_window_p).apply(
    lambda x: pd.Series(x).rank(pct=True).iloc[-1])

# Generate entry and exit signals for Long Strategy with the PctRank and ADX filter
sig = pd.DataFrame(index=train_data.index)
sig['long_entry'] = np.where(
    ((train_data['MACD'] - train_data['MACD_signal']) > 0) & 
    (train_data['ATR_PctRank'] < pctrank_threshold_p) & 
    (train_data['ADX'] > adx_threshold_p), 
    1, 0
)
sig['long_exit'] = np.where(
    (train_data['MACD'] - train_data['MACD_signal']) < 0, 
    1, 0
)

# Create portfolio and calculate statistics for Long Strategy
pft_long = vbt.Portfolio.from_signals(
    train_data['close'],
    entries=sig['long_entry'],
    exits=sig['long_exit'],
    freq='h',
    fees=0.0004
)

# Output portfolio statistics for Long Strategy
pft_long_stats = pft_long.stats()
print("Long Strategy Stats:")
print(pft_long_stats)


In [ ]:
# Retrieve best parameters from the Optuna study for Long Strategy
macd_fast_p = study_calmar.best_params['macd_fast']  
macd_slow_p = study_calmar.best_params['macd_slow']
macd_signal_p = study_calmar.best_params['macd_signal']
atr_period_p = study_calmar.best_params['atr_period']
pctrank_window_p = study_calmar.best_params['pctrank_window']
pctrank_threshold_p = study_calmar.best_params['pctrank_threshold']
adx_period_p = study_calmar.best_params['adx_period']
adx_threshold_p = study_calmar.best_params['adx_threshold']

# Calculate MACD indicators using the best parameters
test_data['MACD'], test_data['MACD_signal'], test_data['MACD_hist'] = ta.MACD(
    test_data['close'], fastperiod=macd_fast_p, slowperiod=macd_slow_p, signalperiod=macd_signal_p)

# Calculate ATR (Average True Range) using the best parameters
test_data['ATR'] = ta.ATR(test_data['high'], test_data['low'], test_data['close'], timeperiod=atr_period_p)

# Calculate ADX (Average Directional Index) using the best parameters
test_data['ADX'] = ta.ADX(test_data['high'], test_data['low'], test_data['close'], timeperiod=adx_period_p)

# Calculate PctRank of ATR over the optimized rolling window
test_data['ATR_PctRank'] = test_data['ATR'].rolling(window=pctrank_window_p).apply(
    lambda x: pd.Series(x).rank(pct=True).iloc[-1])

# Generate entry and exit signals for Long Strategy with the PctRank and ADX filter
sig = pd.DataFrame(index=test_data.index)
sig['long_entry'] = np.where(
    ((test_data['MACD'] - test_data['MACD_signal']) > 0) & 
    (test_data['ATR_PctRank'] < pctrank_threshold_p) & 
    (test_data['ADX'] > adx_threshold_p), 
    1, 0
)
sig['long_exit'] = np.where(
    (test_data['MACD'] - test_data['MACD_signal']) < 0, 
    1, 0
)

# Create portfolio and calculate statistics for Long Strategy
pft_long = vbt.Portfolio.from_signals(
    test_data['close'],
    entries=sig['long_entry'],
    exits=sig['long_exit'],
    freq='h',
    fees=0.0004
)

# Output portfolio statistics for Long Strategy
pft_long_stats = pft_long.stats()
print("Long Strategy Stats:")
print(pft_long_stats)


In [ ]:
# Retrieve best parameters from the Optuna study for Long Strategy
macd_fast_p = study_calmar.best_params['macd_fast']  
macd_slow_p = study_calmar.best_params['macd_slow']
macd_signal_p = study_calmar.best_params['macd_signal']
atr_period_p = study_calmar.best_params['atr_period']
pctrank_window_p = study_calmar.best_params['pctrank_window']
pctrank_threshold_p = study_calmar.best_params['pctrank_threshold']
adx_period_p = study_calmar.best_params['adx_period']
adx_threshold_p = study_calmar.best_params['adx_threshold']

# Calculate MACD indicators using the best parameters
all_data['MACD'], all_data['MACD_signal'], all_data['MACD_hist'] = ta.MACD(
    all_data['close'], fastperiod=macd_fast_p, slowperiod=macd_slow_p, signalperiod=macd_signal_p)

# Calculate ATR (Average True Range) using the best parameters
all_data['ATR'] = ta.ATR(all_data['high'], all_data['low'], all_data['close'], timeperiod=atr_period_p)

# Calculate ADX (Average Directional Index) using the best parameters
all_data['ADX'] = ta.ADX(all_data['high'], all_data['low'], all_data['close'], timeperiod=adx_period_p)

# Calculate PctRank of ATR over the optimized rolling window
all_data['ATR_PctRank'] = all_data['ATR'].rolling(window=pctrank_window_p).apply(
    lambda x: pd.Series(x).rank(pct=True).iloc[-1])

# Generate entry and exit signals for Long Strategy with the PctRank and ADX filter
sig = pd.DataFrame(index=all_data.index)
sig['long_entry'] = np.where(
    ((all_data['MACD'] - all_data['MACD_signal']) > 0) & 
    (all_data['ATR_PctRank'] < pctrank_threshold_p) & 
    (all_data['ADX'] > adx_threshold_p), 
    1, 0
)
sig['long_exit'] = np.where(
    (all_data['MACD'] - all_data['MACD_signal']) < 0, 
    1, 0
)

# Create portfolio and calculate statistics for Long Strategy
pft_long = vbt.Portfolio.from_signals(
    all_data['close'],
    entries=sig['long_entry'],
    exits=sig['long_exit'],
    freq='h',
    fees=0.0004
)

# Output portfolio statistics for Long Strategy
pft_long_stats = pft_long.stats()
print("Long Strategy Stats:")
print(pft_long_stats)


### Max Calmer ( penalize MDD )

In [ ]:
import optuna
import vectorbt as vbt
import pandas as pd
import talib as ta
import numpy as np

# Define the objective function for Optuna optimization (Maximize Calmar Ratio)
def calmar_objective(trial):
    # Suggest values for the parameters
    macd_fast = trial.suggest_int('macd_fast', 12, 26)
    macd_slow = trial.suggest_int('macd_slow', 26, 50)
    macd_signal = trial.suggest_int('macd_signal', 9, 18)
    atr_period = trial.suggest_int('atr_period', 14, 28)  # ATR period suggestion
    pctrank_window = trial.suggest_int('pctrank_window', 10, 50)  # PctRank window suggestion
    pctrank_threshold = trial.suggest_uniform('pctrank_threshold', 0.2, 0.8)  # PctRank threshold suggestion
    adx_period = trial.suggest_int('adx_period', 10, 30)  # ADX period suggestion
    adx_threshold = trial.suggest_uniform('adx_threshold', 20, 40)  # ADX threshold suggestion
    
    # Calculate MACD indicators
    train_data['MACD'], train_data['MACD_signal'], train_data['MACD_hist'] = ta.MACD(
        train_data['close'], fastperiod=macd_fast, slowperiod=macd_slow, signalperiod=macd_signal)

    # Calculate ATR (Average True Range)
    train_data['ATR'] = ta.ATR(train_data['high'], train_data['low'], train_data['close'], timeperiod=atr_period)

    # Calculate ADX (Average Directional Index)
    train_data['ADX'] = ta.ADX(train_data['high'], train_data['low'], train_data['close'], timeperiod=adx_period)

    # Calculate PctRank of ATR over a rolling window
    train_data['ATR_PctRank'] = train_data['ATR'].rolling(window=pctrank_window).apply(
        lambda x: pd.Series(x).rank(pct=True).iloc[-1])

    # Generate signals for long positions with volatility and ADX filter
    sig = pd.DataFrame(index=train_data.index, columns=['long_entry', 'long_exit'])

    # Long entry: MACD conditions, ATR_PctRank < threshold, and ADX > threshold
    sig['long_entry'] = np.where(
        ((train_data['MACD'] - train_data['MACD_signal']) > 0) & 
        (train_data['ATR_PctRank'] < pctrank_threshold) & 
        (train_data['ADX'] > adx_threshold), 
        1, 0
    )

    # Long exit: MACD signal crosses down
    sig['long_exit'] = np.where(
        (train_data['MACD'] - train_data['MACD_signal']) < 0, 
        1, 0
    )

    # Build the portfolio and calculate statistics
    pf1 = vbt.Portfolio.from_signals(
        train_data['close'],
        entries=sig['long_entry'],
        exits=sig['long_exit'],
        freq='h',
        fees=0.0004,
    )
    
    # Calculate necessary statistics manually
    total_return = pf1.total_return()
    max_drawdown = pf1.max_drawdown()

    # Ensure max_drawdown is not zero to avoid division by zero
    if max_drawdown == 0:
        calmar_ratio = 0
    else:
        # Calculate Calmar Ratio (Total Return / Max Drawdown)
        calmar_ratio = total_return / abs(max_drawdown)

    # Return the Calmar Ratio to maximize the objective
    return calmar_ratio

# Run the Optuna optimization to maximize Calmar Ratio
study_calmar = optuna.create_study(direction='maximize')
study_calmar.optimize(calmar_objective, n_trials=500)

# Retrieve and print the best parameters and value for maximizing Calmar Ratio
print(f"Best Parameters to Maximize Calmar Ratio: {study_calmar.best_params}")
print("Best value for Maximized Calmar Ratio:", study_calmar.best_value)


In [ ]:
# Retrieve best parameters from the Optuna study for Long Strategy
macd_fast_p = study_calmar.best_params['macd_fast']  
macd_slow_p = study_calmar.best_params['macd_slow']
macd_signal_p = study_calmar.best_params['macd_signal']
atr_period_p = study_calmar.best_params['atr_period']
pctrank_window_p = study_calmar.best_params['pctrank_window']
pctrank_threshold_p = study_calmar.best_params['pctrank_threshold']
adx_period_p = study_calmar.best_params['adx_period']
adx_threshold_p = study_calmar.best_params['adx_threshold']

# Calculate MACD indicators using the best parameters
train_data['MACD'], train_data['MACD_signal'], train_data['MACD_hist'] = ta.MACD(
    train_data['close'], fastperiod=macd_fast_p, slowperiod=macd_slow_p, signalperiod=macd_signal_p)

# Calculate ATR (Average True Range) using the best parameters
train_data['ATR'] = ta.ATR(train_data['high'], train_data['low'], train_data['close'], timeperiod=atr_period_p)

# Calculate ADX (Average Directional Index) using the best parameters
train_data['ADX'] = ta.ADX(train_data['high'], train_data['low'], train_data['close'], timeperiod=adx_period_p)

# Calculate PctRank of ATR over the optimized rolling window
train_data['ATR_PctRank'] = train_data['ATR'].rolling(window=pctrank_window_p).apply(
    lambda x: pd.Series(x).rank(pct=True).iloc[-1])

# Generate entry and exit signals for Long Strategy with the PctRank and ADX filter
sig = pd.DataFrame(index=train_data.index)
sig['long_entry'] = np.where(
    ((train_data['MACD'] - train_data['MACD_signal']) > 0) & 
    (train_data['ATR_PctRank'] < pctrank_threshold_p) & 
    (train_data['ADX'] > adx_threshold_p), 
    1, 0
)
sig['long_exit'] = np.where(
    (train_data['MACD'] - train_data['MACD_signal']) < 0, 
    1, 0
)

# Create portfolio and calculate statistics for Long Strategy
pft_long = vbt.Portfolio.from_signals(
    train_data['close'],
    entries=sig['long_entry'],
    exits=sig['long_exit'],
    freq='h',
    fees=0.0004
)

# Output portfolio statistics for Long Strategy
pft_long_stats = pft_long.stats()
print("Long Strategy Stats:")
print(pft_long_stats)


In [ ]:
# Retrieve best parameters from the Optuna study for Long Strategy
macd_fast_p = study_calmar.best_params['macd_fast']  
macd_slow_p = study_calmar.best_params['macd_slow']
macd_signal_p = study_calmar.best_params['macd_signal']
atr_period_p = study_calmar.best_params['atr_period']
pctrank_window_p = study_calmar.best_params['pctrank_window']
pctrank_threshold_p = study_calmar.best_params['pctrank_threshold']
adx_period_p = study_calmar.best_params['adx_period']
adx_threshold_p = study_calmar.best_params['adx_threshold']

# Calculate MACD indicators using the best parameters
test_data['MACD'], test_data['MACD_signal'], test_data['MACD_hist'] = ta.MACD(
    test_data['close'], fastperiod=macd_fast_p, slowperiod=macd_slow_p, signalperiod=macd_signal_p)

# Calculate ATR (Average True Range) using the best parameters
test_data['ATR'] = ta.ATR(test_data['high'], test_data['low'], test_data['close'], timeperiod=atr_period_p)

# Calculate ADX (Average Directional Index) using the best parameters
test_data['ADX'] = ta.ADX(test_data['high'], test_data['low'], test_data['close'], timeperiod=adx_period_p)

# Calculate PctRank of ATR over the optimized rolling window
test_data['ATR_PctRank'] = test_data['ATR'].rolling(window=pctrank_window_p).apply(
    lambda x: pd.Series(x).rank(pct=True).iloc[-1])

# Generate entry and exit signals for Long Strategy with the PctRank and ADX filter
sig = pd.DataFrame(index=test_data.index)
sig['long_entry'] = np.where(
    ((test_data['MACD'] - test_data['MACD_signal']) > 0) & 
    (test_data['ATR_PctRank'] < pctrank_threshold_p) & 
    (test_data['ADX'] > adx_threshold_p), 
    1, 0
)
sig['long_exit'] = np.where(
    (test_data['MACD'] - test_data['MACD_signal']) < 0, 
    1, 0
)

# Create portfolio and calculate statistics for Long Strategy
pft_long = vbt.Portfolio.from_signals(
    test_data['close'],
    entries=sig['long_entry'],
    exits=sig['long_exit'],
    freq='h',
    fees=0.0004
)

# Output portfolio statistics for Long Strategy
pft_long_stats = pft_long.stats()
print("Long Strategy Stats:")
print(pft_long_stats)


In [ ]:
# Retrieve best parameters from the Optuna study for Long Strategy
macd_fast_p = study_calmar.best_params['macd_fast']  
macd_slow_p = study_calmar.best_params['macd_slow']
macd_signal_p = study_calmar.best_params['macd_signal']
atr_period_p = study_calmar.best_params['atr_period']
pctrank_window_p = study_calmar.best_params['pctrank_window']
pctrank_threshold_p = study_calmar.best_params['pctrank_threshold']
adx_period_p = study_calmar.best_params['adx_period']
adx_threshold_p = study_calmar.best_params['adx_threshold']

# Calculate MACD indicators using the best parameters
all_data['MACD'], all_data['MACD_signal'], all_data['MACD_hist'] = ta.MACD(
    all_data['close'], fastperiod=macd_fast_p, slowperiod=macd_slow_p, signalperiod=macd_signal_p)

# Calculate ATR (Average True Range) using the best parameters
all_data['ATR'] = ta.ATR(all_data['high'], all_data['low'], all_data['close'], timeperiod=atr_period_p)

# Calculate ADX (Average Directional Index) using the best parameters
all_data['ADX'] = ta.ADX(all_data['high'], all_data['low'], all_data['close'], timeperiod=adx_period_p)

# Calculate PctRank of ATR over the optimized rolling window
all_data['ATR_PctRank'] = all_data['ATR'].rolling(window=pctrank_window_p).apply(
    lambda x: pd.Series(x).rank(pct=True).iloc[-1])

# Generate entry and exit signals for Long Strategy with the PctRank and ADX filter
sig = pd.DataFrame(index=all_data.index)
sig['long_entry'] = np.where(
    ((all_data['MACD'] - all_data['MACD_signal']) > 0) & 
    (all_data['ATR_PctRank'] < pctrank_threshold_p) & 
    (all_data['ADX'] > adx_threshold_p), 
    1, 0
)
sig['long_exit'] = np.where(
    (all_data['MACD'] - all_data['MACD_signal']) < 0, 
    1, 0
)

# Create portfolio and calculate statistics for Long Strategy
pft_long = vbt.Portfolio.from_signals(
    all_data['close'],
    entries=sig['long_entry'],
    exits=sig['long_exit'],
    freq='h',
    fees=0.0004
)

# Output portfolio statistics for Long Strategy
pft_long_stats = pft_long.stats()
print("Long Strategy Stats:")
print(pft_long_stats)


 ### MACD +  RVI  ( max Sharpe  ) 
- RVI 通常先行 MACD
- 多單進場邏輯： 當RVI 指標當中，當RVI 由ｔ期 main line < signal line 變為 ｔ+ 1 期 main line > signal line 時，且 MACD 狀態為慢線> 快線時， 先買入（做多）0.4 * 倉位 

- 加碼： 當RVI 狀態 main line > Signal line , 且 MACD 由t 期 慢線 < 快線變為 t + 1 期快線> 慢線時，加倉買入 0.6 * 倉位 

- 平倉： 當MACD 慢線 > 快線時，平倉所有倉位

In [20]:
import optuna
import vectorbt as vbt
import pandas as pd
import talib as ta
import numpy as np

# Define the objective function for Optuna optimization (Maximize Calmar Ratio)
def calmar_objective(trial):
    # Suggest values for the parameters
    macd_fast = trial.suggest_int('macd_fast', 12, 26)
    macd_slow = trial.suggest_int('macd_slow', 26, 50)
    macd_signal = trial.suggest_int('macd_signal', 9, 18)
    rvi_period = trial.suggest_int('rvi_period', 10, 20)  # RVI period suggestion
    rvi_signal_window = trial.suggest_int('rvi_signal_window', 5, 20)  # RVI signal window suggestion
    entry_weight = trial.suggest_uniform('entry_weight', 0.1, 0.5)  # Weight for initial entry position
    add_weight = trial.suggest_uniform('add_weight', 0.5, 0.9)  # Weight for adding position

    # Calculate MACD using EMA for fast, slow, and signal lines
    ema_fast = ta.EMA(train_data['close'], timeperiod=macd_fast)
    ema_slow = ta.EMA(train_data['close'], timeperiod=macd_slow)
    macd_line = ema_fast - ema_slow
    macd_signal_line = ta.EMA(macd_line, timeperiod=macd_signal)

    train_data['MACD'] = macd_line
    train_data['MACD_signal'] = macd_signal_line
    train_data['MACD_hist'] = macd_line - macd_signal_line

    # Calculate RVI (Relative Volatility Index)
    train_data['RVI'] = ta.RSI(train_data['close'], timeperiod=rvi_period)
    train_data['RVI_Signal'] = train_data['RVI'].rolling(window=rvi_signal_window).mean()

    # Generate signals for long positions
    sig = pd.DataFrame(index=train_data.index)

    # Initial long entry: RVI crosses above its signal and MACD slow line < fast line
    sig['long_entry'] = np.where(
        (train_data['RVI'].shift(1) < train_data['RVI_Signal'].shift(1)) &
        (train_data['RVI'] > train_data['RVI_Signal']) &
        (train_data['MACD'] < train_data['MACD_signal']),
        entry_weight, 0  # Initial entry with optimized weight
    )

    # Add position: MACD crosses above its signal and RVI > RVI signal
    sig['add_position'] = np.where(
        (train_data['RVI'] > train_data['RVI_Signal']) &
        (train_data['MACD'].shift(1) < train_data['MACD_signal'].shift(1)) &
        (train_data['MACD'] > train_data['MACD_signal']),
        add_weight, 0  # Add position with optimized weight
    )

    # Combine the long_entry and add_position signals
    sig['long_entry_combined'] = sig['long_entry'] + sig['add_position']

    # Long exit: MACD slow line crosses above fast line and RVI crosses below its signal
    sig['long_exit'] = np.where(
        (train_data['RVI'] < train_data['RVI_Signal']) &
        (train_data['MACD'] > train_data['MACD_signal']),
        1, 0  # Exit all positions
    )

    # Build the portfolio and calculate statistics
    pf1 = vbt.Portfolio.from_signals(
        train_data['close'],
        entries=sig['long_entry_combined'] > 0,  # Entry signal when combined value is positive
        exits=sig['long_exit'] > 0,  # Exit signal
        size=sig['long_entry_combined'],  # Size based on combined entry signal
        freq='h',
        fees=0.0004,
    )

    # Calculate necessary statistics manually
    total_return = pf1.total_return()
    max_drawdown = pf1.max_drawdown()

    # Ensure max_drawdown is not zero to avoid division by zero
    if max_drawdown == 0:
        calmar_ratio = 0
    else:
        # Calculate Calmar Ratio (Total Return / Max Drawdown)
        calmar_ratio = total_return / abs(max_drawdown)

    # Return the Calmar Ratio to maximize the objective
    return calmar_ratio

# Run the Optuna optimization to maximize Calmar Ratio
study_calmar = optuna.create_study(direction='maximize')
study_calmar.optimize(calmar_objective, n_trials=600)

# Retrieve and print the best parameters and value for maximizing Calmar Ratio
print(f"Best Parameters to Maximize Calmar Ratio: {study_calmar.best_params}")
print("Best value for Maximized Calmar Ratio:", study_calmar.best_value)


[I 2024-08-18 18:04:20,533] A new study created in memory with name: no-name-49c17dd3-3232-4eed-a15e-5f8702375ea8
[I 2024-08-18 18:04:20,690] Trial 0 finished with value: 1.411750795330812 and parameters: {'macd_fast': 15, 'macd_slow': 35, 'macd_signal': 18, 'rvi_period': 15, 'rvi_signal_window': 14, 'entry_weight': 0.47710828540067685, 'add_weight': 0.6307921207487384}. Best is trial 0 with value: 1.411750795330812.
[I 2024-08-18 18:04:20,708] Trial 1 finished with value: 1.8924468813581157 and parameters: {'macd_fast': 14, 'macd_slow': 45, 'macd_signal': 12, 'rvi_period': 15, 'rvi_signal_window': 14, 'entry_weight': 0.3783326494460554, 'add_weight': 0.713916864043735}. Best is trial 1 with value: 1.8924468813581157.
[I 2024-08-18 18:04:20,732] Trial 2 finished with value: -0.16500413037447245 and parameters: {'macd_fast': 12, 'macd_slow': 37, 'macd_signal': 11, 'rvi_period': 11, 'rvi_signal_window': 8, 'entry_weight': 0.3669593049934049, 'add_weight': 0.7507455599904276}. Best is tri

Best Parameters to Maximize Calmar Ratio: {'macd_fast': 14, 'macd_slow': 31, 'macd_signal': 15, 'rvi_period': 20, 'rvi_signal_window': 20, 'entry_weight': 0.4006946546534958, 'add_weight': 0.6647720264034529}
Best value for Maximized Calmar Ratio: 13.097898947837916


In [17]:
# Retrieve best parameters from the Optuna study for Long Strategy
macd_fast_p = study_sharpe.best_params['macd_fast']  
macd_slow_p = study_sharpe.best_params['macd_slow']
macd_signal_p = study_sharpe.best_params['macd_signal']
rvi_period_p = study_sharpe.best_params['rvi_period']

# Calculate MACD indicators using the best parameters
train_data['MACD'], train_data['MACD_signal'], train_data['MACD_hist'] = ta.MACD(
    train_data['close'], fastperiod=macd_fast_p, slowperiod=macd_slow_p, signalperiod=macd_signal_p)

# Calculate RVI (Relative Volatility Index) and its signal line using the best parameters
train_data['RVI'] = ta.RSI(train_data['close'], timeperiod=rvi_period_p)  # RVI calculation
train_data['RVI_Signal'] = train_data['RVI'].rolling(window=3).mean()  # Signal line is a 3-period SMA of RVI

# Generate entry, add position, and exit signals based on the strategy logic
sig = pd.DataFrame(index=train_data.index)

# Long entry: RVI crosses above its signal and MACD slow line < fast line
sig['long_entry'] = np.where(
    (train_data['RVI'].shift(1) < train_data['RVI_Signal'].shift(1)) &
    (train_data['RVI'] > train_data['RVI_Signal']) &
    (train_data['MACD'] < train_data['MACD_signal']),
    1, 0  # Entry signal
)

# Add position: MACD crosses above its signal and RVI > RVI signal
sig['add_position'] = np.where(
    (train_data['RVI'] > train_data['RVI_Signal']) &
    (train_data['MACD'].shift(1) < train_data['MACD_signal'].shift(1)) &
    (train_data['MACD'] > train_data['MACD_signal']),
    1, 0  # Additional entry signal for adding position
)

# Combine the long entry and add position into one signal
sig['long_entry_combined'] = sig['long_entry'] * 0.4 + sig['add_position'] * 0.6

# Long exit: MACD slow line crosses above the fast line
sig['long_exit'] = np.where(
    (train_data['MACD'] > train_data['MACD_signal']),
    1, 0  # Exit signal
)

# Build the portfolio and calculate statistics
pft_long = vbt.Portfolio.from_signals(
    train_data['close'],
    entries=sig['long_entry_combined'] > 0,  # Use combined entry signal
    exits=sig['long_exit'] > 0,  # Exit signal
    size=sig['long_entry_combined'],  # Position size based on combined signal
    freq='h',
    fees=0.0004,
)

# Output portfolio statistics for Long Strategy
pft_long_stats = pft_long.stats()
print("Long Strategy Stats:")
print(pft_long_stats)

# Print total return and max drawdown for debugging Calmar Ratio
total_return = pft_long.total_return()
max_drawdown = pft_long.max_drawdown()

print(f"Total Return: {total_return}")
print(f"Max Drawdown: {max_drawdown}")

# Calculate Calmar Ratio
if max_drawdown != 0:
    calmar_ratio = total_return / abs(max_drawdown)
else:
    calmar_ratio = 0

print(f"Calmar Ratio: {calmar_ratio}")


Long Strategy Stats:
Start                                                 0
End                                               23279
Period                                970 days 00:00:00
Start Value                                       100.0
End Value                                     72.135838
Total Return [%]                             -27.864162
Benchmark Return [%]                         179.458416
Max Gross Exposure [%]                            100.0
Total Fees Paid                               41.083589
Max Drawdown [%]                              61.976024
Max Drawdown Duration                 667 days 12:00:00
Total Trades                                        606
Total Closed Trades                                 605
Total Open Trades                                     1
Open Trade PnL                                 0.142542
Win Rate [%]                                  63.801653
Best Trade [%]                                 7.776199
Worst Trade [%]            

### Revision 

In [21]:
import optuna
import vectorbt as vbt
import pandas as pd
import talib as ta
import numpy as np

# Define the objective function for Optuna optimization (Maximize Calmar Ratio)
def calmar_objective(trial):
    # Suggest values for the parameters
    macd_fast = trial.suggest_int('macd_fast', 12, 26)
    macd_slow = trial.suggest_int('macd_slow', 26, 50)
    macd_signal = trial.suggest_int('macd_signal', 9, 18)
    rvi_period = trial.suggest_int('rvi_period', 10, 20)  # RVI period suggestion
    rvi_signal_window = trial.suggest_int('rvi_signal_window', 5, 20)  # RVI signal window suggestion
    entry_weight = trial.suggest_uniform('entry_weight', 0.1, 0.5)  # Weight for initial entry position
    add_weight = trial.suggest_uniform('add_weight', 0.5, 0.9)  # Weight for adding position

    # Calculate MACD using EMA for fast, slow, and signal lines
    ema_fast = ta.EMA(train_data['close'], timeperiod=macd_fast)
    ema_slow = ta.EMA(train_data['close'], timeperiod=macd_slow)
    macd_line = ema_fast - ema_slow
    macd_signal_line = ta.EMA(macd_line, timeperiod=macd_signal)
    
    train_data['MACD'] = macd_line
    train_data['MACD_signal'] = macd_signal_line
    train_data['MACD_hist'] = macd_line - macd_signal_line

    # Calculate RVI (Relative Volatility Index)
    train_data['RVI'] = ta.RSI(train_data['close'], timeperiod=rvi_period)
    train_data['RVI_Signal'] = train_data['RVI'].rolling(window=rvi_signal_window).mean()

    # Generate signals for long positions
    sig = pd.DataFrame(index=train_data.index)

    # Initial long entry: RVI crosses above its signal and MACD crosses above within t+2 periods
    rvi_cross_up = (train_data['RVI'].shift(1) < train_data['RVI_Signal'].shift(1)) & \
                   (train_data['RVI'] > train_data['RVI_Signal'])

    macd_cross_up_within_t2 = (
        (train_data['MACD'].shift(1) < train_data['MACD_signal'].shift(1)) & 
        (train_data['MACD'].rolling(window=3).apply(lambda x: any(np.diff(x) > 0), raw=True) > 0)
    )

    sig['long_entry'] = np.where(
        rvi_cross_up & macd_cross_up_within_t2,
        entry_weight, 0  # Initial entry with optimized weight
    )

    # Add position: MACD crosses above its signal and RVI > RVI signal
    sig['add_position'] = np.where(
        (train_data['RVI'] > train_data['RVI_Signal']) &
        (train_data['MACD'].shift(1) < train_data['MACD_signal'].shift(1)) &
        (train_data['MACD'] > train_data['MACD_signal']),
        add_weight, 0  # Add position with optimized weight
    )

    # Combine the long_entry and add_position signals
    sig['long_entry_combined'] = sig['long_entry'] + sig['add_position']

    # Long exit: MACD crosses below its signal
    sig['long_exit'] = np.where(
        (train_data['MACD'].shift(1) > train_data['MACD_signal'].shift(1)) & 
        (train_data['MACD'] < train_data['MACD_signal']),
        1, 0  # Exit all positions
    )

    # Build the portfolio and calculate statistics
    pf1 = vbt.Portfolio.from_signals(
        train_data['close'],
        entries=sig['long_entry_combined'] > 0,  # Entry signal when combined value is positive
        exits=sig['long_exit'] > 0,  # Exit signal
        size=sig['long_entry_combined'],  # Size based on combined entry signal
        freq='h',
        fees=0.0004,
    )

    # Calculate necessary statistics manually
    total_return = pf1.total_return()
    max_drawdown = pf1.max_drawdown()

    # Ensure max_drawdown is not zero to avoid division by zero
    if max_drawdown == 0:
        calmar_ratio = 0
    else:
        # Calculate Calmar Ratio (Total Return / Max Drawdown)
        calmar_ratio = total_return / abs(max_drawdown)

    # Return the Calmar Ratio to maximize the objective
    return calmar_ratio

# Run the Optuna optimization to maximize Calmar Ratio
study_calmar = optuna.create_study(direction='maximize')
study_calmar.optimize(calmar_objective, n_trials=600)

# Retrieve and print the best parameters and value for maximizing Calmar Ratio
print(f"Best Parameters to Maximize Calmar Ratio: {study_calmar.best_params}")
print("Best value for Maximized Calmar Ratio:", study_calmar.best_value)


[I 2024-08-18 22:56:59,935] A new study created in memory with name: no-name-7f88e2a3-c222-4e60-8cf9-0dacc4caf4c9
[I 2024-08-18 22:57:00,121] Trial 0 finished with value: 2.4060857959717126 and parameters: {'macd_fast': 22, 'macd_slow': 32, 'macd_signal': 17, 'rvi_period': 19, 'rvi_signal_window': 14, 'entry_weight': 0.40802813797290494, 'add_weight': 0.6639119122312338}. Best is trial 0 with value: 2.4060857959717126.
[I 2024-08-18 22:57:00,188] Trial 1 finished with value: 4.501170599588761 and parameters: {'macd_fast': 20, 'macd_slow': 32, 'macd_signal': 13, 'rvi_period': 16, 'rvi_signal_window': 12, 'entry_weight': 0.2153261504195657, 'add_weight': 0.8975396320025226}. Best is trial 1 with value: 4.501170599588761.
[I 2024-08-18 22:57:00,258] Trial 2 finished with value: 1.9939722994148394 and parameters: {'macd_fast': 20, 'macd_slow': 36, 'macd_signal': 16, 'rvi_period': 15, 'rvi_signal_window': 15, 'entry_weight': 0.35664631234567634, 'add_weight': 0.8361637504521993}. Best is tr

Best Parameters to Maximize Calmar Ratio: {'macd_fast': 16, 'macd_slow': 40, 'macd_signal': 13, 'rvi_period': 13, 'rvi_signal_window': 10, 'entry_weight': 0.2594331528612868, 'add_weight': 0.64773164520891}
Best value for Maximized Calmar Ratio: 11.647631050693558
